In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Normalization

from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

2023-06-18 08:34:14.698208: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 08:34:15.045135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 08:34:15.047000: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 08:34:16.625688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label = 'train' + exp_name)
    ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 5)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_ylim(0., 1.0)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
X_train = pd.read_csv("../raw_data/embeded_data/embedding_train.csv", dtype="float16")

normalizer = Normalization(axis=-1)

X_train = normalizer.fit_transform(X_train)

y_train = pd.read_csv("../raw_data/train_y.csv", delimiter=" ", index_col="Id")['Genre_id']

y_train_cat = to_categorical(y_train, num_classes=30)

del y_train

In [ ]:
def initialize_model():
    model = Sequential()
    
    model.add(layers.Dense(10000, activation="relu", input_dim=26889))
    
    model.add(layers.Dense(1000, activation="relu"))
    
    model.add(layers.Dense(100, activation="relu"))
    
    model.add(layers.Dense(30, activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

    return model

In [ ]:
model_cnn = initialize_model()

es = EarlyStopping(patience=10, restore_best_weights=True, monitor="val_accuracy")

history = model_cnn.fit(X_train, y_train_cat,
              epochs=1000,
              batch_size=32,
              validation_split=0.3,
              callbacks=[es]
             )

del X_train, y_train_cat


X_test = pd.read_csv("../raw_data/embeded_data/embedding_test.csv")
X_test = normalizer.transform(X_test)

y_test = pd.read_csv("../raw_data/test_y.csv", delimiter=" ", index_col="Id")['Genre_id']
y_test_cat = to_categorical(y_test, num_classes=30)

del y_test

res = model_cnn.evaluate(X_test, y_test_cat, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
plot_history(history)